In [ ]:
import numpy as np
import holoviews as hv
hv.extension('matplotlib')

from solversuperres import *

In [ ]:
plots = dict()

In [ ]:
color_range = np.arange(127, -1, -1)

red_cmap = list(map(lambda c: f'#ff{c:02x}{c:02x}', color_range))
green_cmap = list(map(lambda c: f'#{c:02x}ff{c:02x}', color_range))
blue_cmap = list(map(lambda c: f'#{c:02x}{c:02x}ff', color_range))

diverging_cmap = green_cmap[::-1] + blue_cmap[1:]

In [ ]:
k = 10
s_true = Signal_true(d=3, k=k, m=k * 40, eps_dist_spikes=0.25)
s_true.init_spikes(manual_a=np.random.rand(s_true.k) * 4 + 1)

s_esti = s_true.create_estimation(coef_k=5)

In [ ]:
solver = Solver(s_true, s_esti)

In [ ]:
solver.s_true.init_mesures(c=0.25)

In [ ]:
data_true = np.concatenate((s_true.t, s_true.a[:, None]), axis=-1)
s_true_3D = hv.Scatter3D(data_true, vdims=['amps']).opts(
    hv.opts.Scatter3D(marker='x', s=300, color='amps', cmap=red_cmap, fig_size=300))

plots['s_true_3D'] = s_true_3D
hv.output(s_true_3D)

## COMP

In [ ]:
%%time
solver.initialize(opt='COMP', minimizer='cg', tau={'min': -7, 'max': 0}, nit=500, nb_tests=20)

In [ ]:
data_esti = np.concatenate((solver.s_esti.t, solver.s_esti.a[:, None]), axis=-1)
COMP_s_esti_3D = hv.Scatter3D(data_esti, vdims=['amps']).opts(
    hv.opts.Scatter3D(marker='+', s=300, color='amps', cmap=blue_cmap, fig_size=300))

plots['COMP_s_esti_3D'] = COMP_s_esti_3D
hv.output(COMP_s_esti_3D)

In [ ]:
COMP_s_true_esti_3D = s_true_3D * COMP_s_esti_3D
plots['COMP_s_true_esti_3D'] = COMP_s_true_esti_3D
hv.output(COMP_s_true_esti_3D)

In [ ]:
COMP_error = plot.error(solver, alg='COMP', logy=True).opts(aspect=0.5, fontscale=2)
plots['COMP_error'] = COMP_error
hv.output(COMP_error)

## SCOMP

In [ ]:
%%time
solver.initialize(opt='SCOMP', minimizer='cg', tau={'min': -7, 'max': 0}, min_iter=30)

In [ ]:
data_esti = np.concatenate((solver.s_esti.t, solver.s_esti.a[:, None]), axis=-1)
SCOMP_s_init_3D = hv.Scatter3D(data_esti, vdims=['amps']).opts(
    hv.opts.Scatter3D(marker='+', s=300, color='amps', cmap=diverging_cmap, 
                      symmetric=True, fig_size=300, xlim=(0, 1), ylim=(0, 1), zlim=(0, 1)))

plots['SCOMP_s_init_3D'] = SCOMP_s_init_3D
hv.output(SCOMP_s_init_3D)

In [ ]:
SCOMP_s_true_init_3D = s_true_3D * SCOMP_s_init_3D
plots['SCOMP_s_true_init_3D'] = SCOMP_s_true_init_3D
hv.output(SCOMP_s_true_init_3D)

In [ ]:
SCOMP_error_init = plot.error(solver, alg='SCOMP', logy=True).opts(aspect=0.5, fontscale=2)
plots['SCOMP_error_init'] = SCOMP_error_init
hv.output(SCOMP_error_init)

In [ ]:
SCOMP_condition_init = plot.condition(solver, alg='SCOMP').opts(fontscale=2, aspect=1)
plots['SCOMP_condition_init'] = SCOMP_condition_init
hv.output(SCOMP_condition_init)

In [ ]:
%%time
solver.optimize(eps_proj=0.20, project=True, tau={'min': -8, 'max': 1}, nit=3_000)

In [ ]:
data_esti = np.concatenate((solver.s_esti.t, solver.s_esti.a[:, None]), axis=-1)
SCOMP_s_esti_3D = hv.Scatter3D(data_esti, vdims=['amps'], group='esti').opts(
    hv.opts.Scatter3D(marker='+', s=300, color='amps', cmap=diverging_cmap, 
                      symmetric=True, fig_size=300, xlim=(0, 1), ylim=(0, 1), zlim=(0, 1)))
hv.output(SCOMP_s_esti_3D)

In [ ]:
SCOMP_s_true_esti_3D = s_true_3D * SCOMP_s_esti_3D
plots['SCOMP_s_true_esti_3D'] = SCOMP_s_true_esti_3D
hv.output(SCOMP_s_true_esti_3D)

In [ ]:
SCOMP_error_esti = plot.error(solver, alg='PGD', logy=True).opts(aspect=0.5, fontscale=2)
plots['SCOMP_error_esti'] = SCOMP_error_esti
hv.output(SCOMP_error_esti)

## Saving plots

In [ ]:
def save_svg(directory, dict_plots):
    for name, plot in dict_plots.items():
        hv.save(plot, f'{directory}/{name}', fmt='svg')

In [ ]:
save_svg('imgs', plots)